In [ ]:
import numpy as np

In [ ]:
# The following routines are for 32-bits
# TODO: have onese for 8, 16, and 64 bits

def part1by1(n):
    n&= 0x0000ffff
    n = (n | (n << 8)) & 0x00FF00FF
    n = (n | (n << 4)) & 0x0F0F0F0F
    n = (n | (n << 2)) & 0x33333333
    n = (n | (n << 1)) & 0x55555555
    return n

def unpart1by1(n):
    n&= 0x55555555
    n = (n ^ (n >> 1)) & 0x33333333
    n = (n ^ (n >> 2)) & 0x0f0f0f0f
    n = (n ^ (n >> 4)) & 0x00ff00ff
    n = (n ^ (n >> 8)) & 0x0000ffff
    return n

def interleave2(x, y):
    return part1by1(x) | (part1by1(y) << 1)

def deinterleave2(n):
    return unpart1by1(n), unpart1by1(n >> 1)

In [ ]:
# The following routines are for 32-bits
# TODO: have onese for 8, 16, and 64 bits

def part1by2(n):
    n&= 0x000003ff
    n = (n ^ (n << 16)) & 0xff0000ff
    n = (n ^ (n <<  8)) & 0x0300f00f
    n = (n ^ (n <<  4)) & 0x030c30c3
    n = (n ^ (n <<  2)) & 0x09249249
    return n


def unpart1by2(n):
    n&= 0x09249249
    n = (n ^ (n >>  2)) & 0x030c30c3
    n = (n ^ (n >>  4)) & 0x0300f00f
    n = (n ^ (n >>  8)) & 0xff0000ff
    n = (n ^ (n >> 16)) & 0x000003ff
    return n

def interleave3(x, y, z):
    return part1by2(x) | (part1by2(y) << 1) | (part1by2(z) << 2)

def deinterleave3(n):
    return unpart1by2(n), unpart1by2(n >> 1), unpart1by2(n >> 2)

In [ ]:
#S,R,C = 15,15,15
S,R,C = 2,2,2
S,R,C = 5,3,1

n = 3

In [ ]:
# overwrite the LUT with membrance centric radix generated

LUT_ALPHA_STAR        = np.loadtxt(f'LUT_{n}D_alpha_star.tsv',            dtype=np.int32)                    # 48 x 4
LUT_DD                = np.loadtxt(f'LUT_{n}D_alpha_{n-1}_scenarios.tsv', dtype=np.int32).reshape(-1,3,n+1)  # 48 x 3 x 4
LUT_PosDartDescriptor = np.loadtxt(f'LUT_{n}D_PosDartDescriptor.tsv',     dtype=np.uint8, skiprows=0)        # 48 x 4     

In [ ]:
# alphas for i != n-1

def alpha_i (i,d):
    
    i48 =   d & 0b111111 
    
    x,y,z = deinterleave3(d      >> 6)  
    return (  interleave3(x,y,z) << 6) | LUT_ALPHA_STAR [i48,i]

In [ ]:
 def alpha_2 (d):
    """
    Implicitly encoded alpha_2
    
    Steps
        1. get 48-index of the dart for that voxel
        2. get  3-index determining the scenario
        3. zcode -> [x,y,z] + [dx,dy,dz]
        4. update the z-core, append new 6-bits, return
    """
    i48 =   d & 0b111111                                                       # dart     index
    
    # ##############
    # scenatio idnex based on dimension and positional dart descriptors
    m = (S,R,C)                                  # dimensions
    p = deinterleave3(d      >> 6) [::-1]        # position in the z,y,x order
    x_n1, I_n1, x_n2, I_n2 = LUT_PosDartDescriptor [i48,:4]   # first 4 descriptors

    # preparing for worst: internal scenario:
    s = 2

    if p[x_n1] == m [x_n1] * (1 - I_n1):
#         if I_n1 == 0 and p[x_n1] == m [x_n1] or \
#            I_n1 == 1 and p[x_n1] == 0:
        s = 1

        if p[x_n2] == I_n2 * (m[x_n2]-1):
#             if I_n2 == 0 and p[x_n2] == 0         or \
#                I_n2 == 1 and p[x_n2] == m[x_n2]-1 :
                s = 0
    i3 = s
    
#   dx,dy,dz,bbbbbb = LUT_DD [i48,i3]  # the order of the deltas in the table is dz,dy,dx
    x,y,z = deinterleave3(d      >> 6) + LUT_DD [i48,i3,:3][::-1]  # deinterleave the z-code and update it by dx,dy,dz
    return (  interleave3(x,y,z) << 6) | LUT_DD [i48,i3, 3]        # interleave and append bbbbbb

In [ ]:
alpha_0 = lambda d: alpha_i (0,d) # alpha_0 = lambda d: d ^ 0b000001
alpha_1 = lambda d: alpha_i (1,d) # can still be done by bit tricks
alpha_3 = lambda d: alpha_i (3,d) # alpha_3 = lambda d: d ^ 0b001000

## $D$: Set of darts

In [ ]:
# set of darts

D = set()

for z in range (S):
    for y in range (R):
        for x in range (C):
            for i in range (48):
                D |= {interleave3 (x,y,z) << 6 | i}

z = S
for y in range (R):
    for x in range (C):
        for i in range (16):
            D |= {interleave3 (x,y,z) << 6 | i}

y = R
for x in range (C):
    for z in range (S):
        for i in range (16):
            D |= {interleave3 (x,y,z) << 6 | 0b010000 | i}
    
x = C
for y in range (R):
    for z in range (S):
        for i in range (16):
            if (interleave3 (x,y,z) << 6 | 0b100000 | i) in D:
                print ('grrr')
            D |= {interleave3 (x,y,z) << 6 | 0b100000 | i}

### involution checks

In [ ]:
# involution checks

assert all ([alpha_0(alpha_0 (d)) == d for d in D])
assert all ([alpha_1(alpha_1 (d)) == d for d in D])
assert all ([alpha_2(alpha_2 (d)) == d for d in D])

assert all ([alpha_0(alpha_2(alpha_0(alpha_2(d)))) == d for d in D])
assert all ([alpha_0(alpha_3(alpha_0(alpha_3(d)))) == d for d in D])
assert all ([alpha_1(alpha_3(alpha_1(alpha_3(d)))) == d for d in D])

### sanity check with array-based gmaps

In [ ]:
from combinatorial.gmaps import nGmap

In [ ]:
A = np.full ((4,1+max(D)),-1)  # initialize with invalid darts (-1)

for d in D:
    A[0,d] = alpha_0(d)
    A[1,d] = alpha_1(d)
    A[2,d] = alpha_2(d)
    A[3,d] = alpha_3(d)

In [ ]:
A

array([[    1,     0,     3, ..., 17676, 17679, 17678],
       [    4,     6,     5, ..., 17674, 17673, 17675],
       [   16,    17,   152, ..., 17443, 17514, 17515],
       [    8,     9,    10, ..., 17669, 17670, 17671]])

In [ ]:
g = nGmap.from_alpha_array (A)
assert g.is_valid
assert g.no_0_cells == (S+1)*(R+1)*(C+1)
assert g.no_1_cells == S*(R+1)*(C+1) + (S+1)*R*(C+1) + (S+1)*(R+1)*C
assert g.no_2_cells == (S+1)*R*C + (R+1)*S*C + (C+1)*S*R == 3*S*R*C + R*C + S*C + S*R
assert g.no_3_cells == S*R*C + 1 # also background counted
assert g.no_ccs     == 1

In [ ]:
print (f'{S}x{R}x{C} volume resulted in', g)

5x3x1 volume resulted in 3-gMap of 1088 darts:
  # 0-cells: 48
  # 1-cells: 100
  # 2-cells: 68
  # 3-cells: 16
  # ccs    : 1



In [ ]:
from IPython.lib.display import Audio

framerate = 4410
play_time_seconds = 1


t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)